In [1]:
import numpy
import pandas as pd
import matplotlib as plt
import numpy as np


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import LabelEncoder,RobustScaler
from sklearn.model_selection import KFold, cross_val_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
newTrain=train[['SalePrice','LotArea','OverallQual','OverallCond']]
pd.options.mode.chained_assignment = None

In [3]:
newTrain['isVeryOld']=train['YearBuilt'].apply(lambda x:1 if x<1945 else 0 )
newTrain['isNormalAge']=train['YearBuilt'].apply(lambda x:1 if (x>=1945&x<2003) else 0 )
newTrain['isVeryNew']=train['YearBuilt'].apply(lambda x:1 if x>2003 else 0 )

In [4]:
X_test=test[['LotArea','OverallQual','OverallCond']]
X_test['isVeryOld']=test['YearBuilt'].apply(lambda x:1 if x<1945 else 0 )
X_test['isNormalAge']=test['YearBuilt'].apply(lambda x:1 if (x>=1945&x<2003) else 0 )
X_test['isVeryNew']=test['YearBuilt'].apply(lambda x:1 if x>2003 else 0 )

In [13]:
X_train=newTrain[[c for c  in newTrain.columns if c not in ['SalePrice']]].values
y_train=newTrain['SalePrice'].ravel()

model=LinearRegression()
X_train

array([[ 8450,     7,     5,     0,     1,     0],
       [ 9600,     6,     8,     0,     1,     0],
       [11250,     7,     5,     0,     1,     0],
       ...,
       [ 9042,     7,     9,     1,     1,     0],
       [ 9717,     5,     6,     0,     1,     0],
       [ 9937,     5,     6,     0,     1,     0]])

In [14]:
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
y_pred=model.predict(X_train)

In [16]:
y_test=model.predict(X_test)

In [17]:
# Calculate metrics
print("Mean Absolute Error: %.2f" %mean_absolute_error(y_train, y_pred))
print("Mean Squared Error: %.2f" %mean_squared_error(y_train, y_pred))
print("Root Mean Squared Error: %.2f" %np.sqrt(mean_squared_error(y_train, y_pred)))
print("Coefficient of Determination: %.2f" %r2_score(y_train, y_pred))

Mean Absolute Error: 31152.89
Mean Squared Error: 2097195291.55
Root Mean Squared Error: 45795.14
Coefficient of Determination: 0.67


In [18]:

subDf=pd.DataFrame({'Id':test['Id'].values})
subDf['SalePrice']=y_test
subDf.to_csv("submission.csv", index=False)